In [ ]:
!pip install keras_core keras_cv munch tensorflow -U

In [ ]:
!ps -ef

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import cv2
import keras.backend as K
import keras_cv
import re
import tqdm
import os
import copy

from tensorflow import keras
from keras_cv import visualization

2023-10-14 00:47:53.440279: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-14 00:47:53.440343: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-14 00:47:53.440396: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using TensorFlow backend


# Main

## Download and extract data

In [2]:
# cub_200_2011_url = 'https://drive.google.com/uc?export=download&id=1hbzc_P1FuxMkcabkgn9ZKinBwW683j45'

cub_200_2011_url = 'https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz'
cub_v2_url = 'https://drive.google.com/u/0/uc?id=1U6cwKHS65wayT9FFvoLIA8cn1k0Ot2M1&export=download'

dl_manager = tfds.download.DownloadManager(
    download_dir='/kaggle/working/dataset',
    extract_dir='/kaggle/working/dataset',
    dataset_name='CUB'
)

data_dirs = dl_manager.download_and_extract([
    cub_200_2011_url,
    cub_v2_url
])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [3]:
!rsync -av /kaggle/working/dataset/TAR.u_0_ucid_1U6cwKHS65wayT9FFvoLI_export_downloadtucVXoFCaF1zLeWZ2WFyzjnY-FlfYz1pOwkzbO6x5Ds/ /kaggle/working/CUB
!rsync -av /kaggle/working/dataset/TAR_GZ.data.calt.edu_reco_65de-vp15_file_CUB_200_ahP8ECcIFAKOrAGhfweAoyw8ALO8u5-8goN4zja5p1g.tgz/CUB_200_2011/images/ /kaggle/working/CUB

sending incremental file list
./
001.Black_footed_Albatross/
002.Laysan_Albatross/
003.Sooty_Albatross/
004.Groove_billed_Ani/
005.Crested_Auklet/
006.Least_Auklet/
007.Parakeet_Auklet/
008.Rhinoceros_Auklet/
009.Brewer_Blackbird/
010.Red_winged_Blackbird/
011.Rusty_Blackbird/
012.Yellow_headed_Blackbird/
013.Bobolink/
014.Indigo_Bunting/
015.Lazuli_Bunting/
016.Painted_Bunting/
017.Cardinal/
018.Spotted_Catbird/
019.Gray_Catbird/
020.Yellow_breasted_Chat/
021.Eastern_Towhee/
022.Chuck_will_Widow/
023.Brandt_Cormorant/
024.Red_faced_Cormorant/
025.Pelagic_Cormorant/
026.Bronzed_Cowbird/
027.Shiny_Cowbird/
028.Brown_Creeper/
029.American_Crow/
030.Fish_Crow/
031.Black_billed_Cuckoo/
032.Mangrove_Cuckoo/
033.Yellow_billed_Cuckoo/
034.Gray_crowned_Rosy_Finch/
035.Purple_Finch/
036.Northern_Flicker/
037.Acadian_Flycatcher/
038.Great_Crested_Flycatcher/
039.Least_Flycatcher/
040.Olive_sided_Flycatcher/
041.Scissor_tailed_Flycatcher/
042.Vermilion_Flycatcher/
043.Yellow_bellied_Flycatcher/
0

In [4]:
%cd /kaggle/working
!git clone https://github.com/clovaai/wsolevaluation.git

/kaggle/working
fatal: destination path 'wsolevaluation' already exists and is not an empty directory.


## Load data

In [5]:
input_shape = (224, 224, 3)
num_classes = 200
batch_size = 32
drop_threshold = 0.8
drop_prob = 0.25
sim_fg_thres = 0.4
sim_bg_thres = 0.2
loss_ratio_sim = 0.5
loss_ratio_norm = 0.05
loss_ratio_drop = 2.0

img_dir='/kaggle/working/CUB'

In [6]:
train_labels = tf.data.TextLineDataset('/kaggle/working/wsolevaluation/metadata/CUB/train/class_labels.txt')
train_ids = tf.data.TextLineDataset('/kaggle/working/wsolevaluation/metadata/CUB/train/image_ids.txt')
train_ds = tf.data.Dataset.zip(train_ids, train_labels)

val_labels = tf.data.TextLineDataset('/kaggle/working/wsolevaluation/metadata/CUB/val/class_labels.txt')
val_ids = tf.data.TextLineDataset('/kaggle/working/wsolevaluation/metadata/CUB/val/image_ids.txt')
val_ds = tf.data.Dataset.zip(val_ids, val_labels)

In [7]:
def preprocess_data(id, label):
    # Load and resize image
    raw = tf.io.read_file(tf.strings.join([img_dir, id], separator='/'))
    image = tf.image.decode_jpeg(raw, channels=3)
    image = tf.image.resize(image, input_shape[:2])

    # Convert index label to binary array [0, 0, 1, ..., 0, 0]
    label =  tf.strings.to_number(tf.strings.split(label, ',')[-1], out_type=tf.int32)
    label = tf.one_hot(label, num_classes)

    return image, {
        'id': id,
        'classes': label
    }

In [8]:
with open('/kaggle/working/wsolevaluation/metadata/CUB/train/image_ids.txt', 'r') as f:
    train_num_examples = len(f.readlines())
    print(train_num_examples)

with open('/kaggle/working/wsolevaluation/metadata/CUB/val/image_ids.txt', 'r') as f:
    val_num_examples = len(f.readlines())
    print(val_num_examples)

with open('/kaggle/working/wsolevaluation/metadata/CUB/test/image_ids.txt', 'r') as f:
    test_num_examples = len(f.readlines())
    print(test_num_examples)



5994
1000
5794


In [9]:
train_ds = train_ds.map(
    preprocess_data,
    num_parallel_calls=tf.data.AUTOTUNE
)
train_ds = train_ds.cache()
train_ds = train_ds.shuffle(train_num_examples)
train_ds = train_ds.batch(batch_size=batch_size)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(
    preprocess_data,
    num_parallel_calls=tf.data.AUTOTUNE
)
val_ds = val_ds.cache()
val_ds = val_ds.batch(batch_size=batch_size)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

## Define layer and loss function

### Attentive dropout layer

In [10]:
class AttentiveDropout(keras.layers.Layer):
    def __init__(self, drop_threshold=0.7, drop_prob=1.):
        super(AttentiveDropout, self).__init__()

        if not (0 <= drop_threshold <= 1):
            raise ValueError("Drop threshold must be in range [0, 1].")

        if not (0 <= drop_prob <= 1):
            raise ValueError("Drop probability must be in range [0, 1].")

        self.drop_threshold = drop_threshold
        self.drop_prob = drop_prob
        self.globalmaxpool = keras.layers.GlobalMaxPooling2D(keepdims=True)

    def call(self, input):
        m_avg = K.mean(input, axis=3, keepdims=True)
        thres = tf.broadcast_to(self.globalmaxpool(m_avg) * self.drop_threshold, K.shape(m_avg))

        dropped_mask = tf.where(m_avg > thres, tf.zeros_like(m_avg), tf.ones_like(m_avg))
        random_tensor = tf.random.uniform(shape=tf.shape(m_avg), dtype=tf.float32)
        dropped_mask = tf.where(random_tensor <= self.drop_prob, dropped_mask, tf.ones_like(dropped_mask))

        erased_input = input * dropped_mask

        return erased_input

### Define loss

In [11]:
@tf.function
def get_sim(feature, weight):
    # Calculate cosine similarity between weight and feature
    x_normalized = K.l2_normalize(feature, axis=3)
    weight_normalized = K.l2_normalize(weight, axis=0)

    # Reshape the normalized weights to have shape (1, 1, num_input_channels, num_output_channels)
    weight_normalized_reshaped = K.reshape(weight_normalized, (1, 1, weight_normalized.shape[0], -1))

    # weight.shape[1] = num_classes in tf
    return keras.layers.Conv2D(filters=weight.shape[1], kernel_size=(1, 1), use_bias=False).convolution_op(x_normalized, weight_normalized_reshaped)

@tf.function
def get_loss_alignment(
        feature,
        weight,
        target,
        sim_bg_thres,
        sim_fg_thres,
        eps=1e-15
    ):
    sim = get_sim(feature, weight)
    # Flatten all class similarity map per sample
    sim_flat = tf.keras.layers.Reshape((-1, num_classes))(sim) # result shape B 784 num_classes

    feature_norm = tf.norm(feature, axis=3)

    # Minmax normalize
    feature_norm_flatten = K.batch_flatten(feature_norm)
    min = K.min(feature_norm_flatten, axis=1)
    max = K.max(feature_norm_flatten, axis=1)
    feature_norm_minmax = K.reshape(
        (feature_norm_flatten - min[:, tf.newaxis]) / (max[:, tf.newaxis] - min[:, tf.newaxis]),
        K.shape(feature_norm)
    )

    feature_norm_minmax_flat = K.batch_flatten(feature_norm_minmax)

    # Sim loss
    sim_fg = K.cast(feature_norm_minmax_flat > 0.4, dtype='float32')
    sim_bg = K.cast(feature_norm_minmax_flat < 0.2, dtype='float32')

    sim_fg_mean = K.sum(sim_fg[..., tf.newaxis] * sim_flat, axis=1) / (K.sum(sim_fg, axis=1) + eps)[:, tf.newaxis]
    sim_bg_mean = K.sum(sim_bg[..., tf.newaxis] * sim_flat, axis=1) / (K.sum(sim_bg, axis=1) + eps)[:, tf.newaxis]

    loss_sim = K.cast(target, dtype='float32') * (sim_bg_mean - sim_fg_mean)

    # per sample loss sim
    loss_sim = K.sum(loss_sim, axis=1) / K.cast(tf.math.count_nonzero(loss_sim, axis=1), dtype='float32')
    # scalar loss sim
    loss_sim = K.mean(loss_sim, axis=-1)

    # Norm loss
    norm_fg = K.cast(sim_flat > 0, dtype='float32')
    norm_bg = K.cast(sim_flat < 0, dtype='float32')

    norm_fg_mean = K.sum(norm_fg * feature_norm_minmax_flat[..., tf.newaxis], axis=1) / (K.sum(norm_fg, axis=1) + eps)
    norm_bg_mean = K.sum(norm_bg * feature_norm_minmax_flat[..., tf.newaxis], axis=1) / (K.sum(norm_bg, axis=1) + eps)

    loss_norm = K.cast(target, dtype='float32') * (norm_bg_mean - norm_fg_mean)

    # per sample loss norm
    loss_norm = K.sum(loss_norm, axis=1) / K.cast(tf.math.count_nonzero(loss_norm, axis=1), dtype='float32')
    # scalar loss norm
    loss_norm = K.mean(loss_norm, axis=-1)

    return loss_sim, loss_norm

## MaxBoxAccV2 callback

In [12]:
%cd /kaggle/working/wsolevaluation
from evaluation import evaluate_wsol

/kaggle/working/wsolevaluation


In [25]:
class EvaluateMaxBoxAccV2Callback(keras.callbacks.Callback):
    def __init__(
        self,
        data,
        scoremap_root,
        metadata_root,
        mask_root,
        dataset_name,
        split,
        cam_curve_interval,
        multi_iou_eval,
        multi_contour_eval,
        iou_threshold_list=[30, 50, 70]
    ):
        self.data = data
        self.scoremap_root = scoremap_root
        self.metadata_root = metadata_root
        self.mask_root = mask_root
        self.dataset_name = dataset_name
        self.split = split
        self.cam_curve_interval = cam_curve_interval
        self.multi_iou_eval = multi_iou_eval
        self.multi_contour_eval = multi_contour_eval
        self.iou_threshold_list = iou_threshold_list

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 5 != 0:
            return
        
        for batch_idx, batch in self.data.enumerate():
            x, y = batch

            y_pred = model(x, return_cam=True, target=y['classes'])

            for i, (image, cams_per_image, prediction, gt, image_id) in enumerate(zip(x, y_pred['cam'], y_pred['classes'], y['classes'], y['id'])):
                shape = image.shape

                prediction = prediction.numpy()

                prediction_rounded = np.flatnonzero(np.round(prediction))

                for target_class, norm_heatmap in cams_per_image.items():
                    # Set up path for saving
                    image_id = image_id.numpy().decode('ASCII')
                    name = os.path.basename(image_id)
                    saved_dir = os.path.join(self.scoremap_root, os.path.dirname(image_id))
                    cam_path = os.path.join(saved_dir, name)

                    if not os.path.exists(os.path.dirname(cam_path)):
                        os.makedirs(os.path.dirname(cam_path))

                    # Save min max normalized heatmap
                    np.save(cam_path, norm_heatmap)

        logs['val_localization'] = evaluate_wsol(
            scoremap_root=self.scoremap_root,
            metadata_root=self.metadata_root,
            mask_root=self.mask_root,
            dataset_name=self.dataset_name,
            split=self.split,
            cam_curve_interval=self.cam_curve_interval,
            multi_iou_eval=self.multi_iou_eval,
            multi_contour_eval=self.multi_contour_eval,
            iou_threshold_list=self.iou_threshold_list
        )

        os.system('rm -rf /kaggle/working/cam')

## Get model

In [26]:
class WSOD_Resnet50(keras.Model):
    def __init__(
            self,
            input_shape,
            num_classes = 1000,
            drop_threshold = 0.7,
            drop_prob = 1.0,
            sim_fg_thres = 0.4,
            sim_bg_thres = 0.2,
            loss_ratio_sim = 0.5,
            loss_ratio_norm = 0.05,
            loss_ratio_drop = 2.0,
        ):
        self.num_classes = num_classes
        self.sim_fg_thres = sim_fg_thres
        self.sim_bg_thres = sim_bg_thres
        self.loss_ratio_sim = loss_ratio_sim
        self.loss_ratio_norm = loss_ratio_norm
        self.loss_ratio_drop = loss_ratio_drop

        super().__init__()
        # Define model
        resnet50 = keras.applications.ResNet50(
            include_top=False,
            weights="imagenet",
            input_tensor=None,
            input_shape=input_shape,
            pooling='None'
        )
        model_config = resnet50.get_config()

        for i, layer_config in enumerate(model_config['layers']):
            if  re.match(r'^conv[45].*conv$', layer_config['config']['name']):
                model_config['layers'][i]['config']['strides'] = (1, 1)

        model = keras.Model.from_config(model_config)
        model.set_weights(resnet50.get_weights())

        self.resnet_first_half = keras.models.Model(
                                model.input,
                                model.get_layer('conv5_block1_out').output,
                            )

        self.att_drop = AttentiveDropout(drop_threshold, drop_prob)

        self.resnet_second_half = keras.models.Model(
                                model.get_layer('conv5_block2_1_conv').input,
                                model.get_layer('conv5_block3_out').output,
                            )
        self.gap = keras.layers.GlobalAveragePooling2D()
        self.classification = keras.layers.Dense(num_classes, activation = 'softmax', name='classification')

    def get_cam(self, input, feature, targets):
        def normalize_minmax(cam):
            if np.isnan(cam).any():
                return np.zeros_like(cam)

            if cam.min() == cam.max():
                return np.zeros_like(cam)

            return (cam - cam.min()) / (cam.max() - cam.min())

        cam_batch = []

        for image, image_feature, target in zip(input, feature, targets):
            shape = image.shape

            target = target.numpy()
            target = np.round(target)

            cams_per_image = {}

            for c in np.flatnonzero(target):
                maps = tf.tensordot(
                    self.classification.weights[0][:, c], # shape (C, )
                    tf.transpose(image_feature, [2, 0, 1]), # shape (H, W, C) -> (C, H, W)
                    1
                )

                heatmap = cv2.resize(maps.numpy(), (shape[1], shape[0]), interpolation = cv2.INTER_CUBIC)

                norm_heatmap = normalize_minmax(heatmap)

                # norm_heatmap = cv2.normalize(heatmap, None, alpha = 1, beta = 0, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)
                # norm_heatmap = norm_heatmap.astype(np.uint8)

                cams_per_image[c] = norm_heatmap

            cam_batch.append(cams_per_image)

        return cam_batch


    def call(self, input, training=False, return_cam=False, target=None):
        # Model pipeline
        x = keras.applications.resnet.preprocess_input(input)
        x = self.resnet_first_half(x, training=training)

        # Temp variable for calculating loss while training
        erased_feature = tf.identity(x)

        # Continue classification branch
        x = self.resnet_second_half(x, training=training)
        feature = tf.identity(x)
        x = self.gap(x)

        class_output = self.classification(x)

        result = {}
        result.update({
            'classes': class_output
        })

        if training:
            # Branch: Consistency with attentive dropout
            erased_feature = self.att_drop(erased_feature)
            erased_feature = self.resnet_second_half(erased_feature, training=training)

            result.update({
                'erased_feature': erased_feature,
                'feature': feature
            })

        if return_cam:
            if target is None:
                raise ValueError('Target is not provided')

            batch_cam = self.get_cam(input, feature, target)

            result.update({
                'cam': batch_cam
            })

        return result

In [27]:
model = WSOD_Resnet50(
            input_shape = input_shape,
            num_classes = num_classes,
            drop_threshold = drop_threshold,
            drop_prob = drop_prob,
            sim_fg_thres = sim_fg_thres,
            sim_bg_thres = sim_bg_thres,
            loss_ratio_sim = loss_ratio_sim,
            loss_ratio_norm = loss_ratio_norm,
            loss_ratio_drop = loss_ratio_drop,
        )
# model.build((1,224,224,3))
model.compile(optimizer='sgd')
# model.compile(
#     optimizer=keras.optimizers.SGD(
#         learning_rate=0.002,
#         momentum=0.9,
#         weight_decay=0.0001,
#         nesterov=True
#     )
# )

## Training

In [28]:
!rm /kaggle/working/temp/*

rm: cannot remove '/kaggle/working/temp/*': No such file or directory


In [29]:
%cd /kaggle/working
!mkdir /kaggle/working/temp

/kaggle/working
mkdir: cannot create directory ‘/kaggle/working/temp’: File exists


### Callbacks

In [30]:
weights_path = '/kaggle/working/temp'
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = 'val_localization',
    mode = 'max',
    min_delta = 0.025,
    patience = 2,
    factor = 0.1,
    verbose = 1,
    cooldown = 0,
    min_lr = 0.00000001
)

cb_early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_localization',
    patience=6,
    min_delta=0.025,
    verbose=1,
    mode='max'
)

cb_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    f'{weights_path}/model.ckpt',
    monitor='val_localization',
    save_best_only=True,
    save_weights_only=True,
    verbose=1,
    mode='max'
)

csv_logger = tf.keras.callbacks.CSVLogger(f'{weights_path}/training.log', append=True)

cb_evaluate_maxboxaccv2 = EvaluateMaxBoxAccV2Callback(
    data=val_ds,
    scoremap_root='/kaggle/working/cam/val',
    metadata_root='/kaggle/working/wsolevaluation/metadata',
    mask_root='/kaggle/working/dataset',
    dataset_name='CUB',
    split='val',
    cam_curve_interval=0.005,
    multi_iou_eval=True,
    multi_contour_eval=True
)

In [31]:
callbacks =  tf.keras.callbacks.CallbackList(
    [
        cb_evaluate_maxboxaccv2,
        cb_checkpoint,
        csv_logger
    ],
    add_history=True, model=model
)

### Optimizers

In [32]:
classifier_optimizer = keras.optimizers.SGD(
    learning_rate=0.002,
    momentum=0.9,
    weight_decay=0.0001,
    nesterov=True
)

feature_extraction_optimizer = keras.optimizers.SGD(
    learning_rate=0.002 * 10,
    momentum=0.9,
    weight_decay=0.0001,
    nesterov=True
)

### Metrics and Losses

In [33]:
cce = keras.losses.CategoricalCrossentropy()
mae = keras.losses.MeanAbsoluteError()

loss_tracker = keras.metrics.Mean(name='loss')

top1_acc_train = keras.metrics.TopKCategoricalAccuracy(k=1, name='top1_acc_train')
top5_acc_train = keras.metrics.TopKCategoricalAccuracy(k=5, name='top5_acc_train')

top1_acc_val = keras.metrics.TopKCategoricalAccuracy(k=1, name='top1_acc_val')
top5_acc_val = keras.metrics.TopKCategoricalAccuracy(k=5, name='top5_acc_val')

### Define train and val function

In [34]:
@tf.function
def train_step(data, warm):
    x, y = data

    with tf.GradientTape() as tape:
        y_pred = model(x, training=True)

        # Calculate loss
        loss_cce = cce(y['classes'], y_pred['classes'])

        loss_drop = mae(y_pred['feature'], y_pred['erased_feature'])

        loss_sim, loss_norm = get_loss_alignment(
            feature=y_pred['feature'],
            weight=model.classification.weights[0],
            target=y['classes'],
            sim_bg_thres=model.sim_bg_thres,
            sim_fg_thres=model.sim_fg_thres,
        )

        if warm:
            loss = loss_cce + \
                    model.loss_ratio_drop * loss_drop
        else:
            loss = loss_cce + \
                    model.loss_ratio_drop * loss_drop + \
                    model.loss_ratio_sim * loss_sim + \
                    model.loss_ratio_norm * loss_norm

    # Get layers' variable
    feature_extraction_variables = model.trainable_variables[:172]
    classifier_variables = model.trainable_variables[172:]

    # Compute gradients
    grads = tape.gradient(loss, feature_extraction_variables + classifier_variables)
    feature_extraction_grads, classifier_grads = grads[:172], grads[172:]
    
    # Update weights
    feature_extraction_optimizer.apply_gradients(zip(feature_extraction_grads, feature_extraction_variables))
    classifier_optimizer.apply_gradients(zip(classifier_grads, classifier_variables))

    # Update metrics (includes the metric that tracks the loss)
    loss_tracker.update_state(loss)

    top1_acc_train.update_state(y['classes'], y_pred['classes'])
    top5_acc_train.update_state(y['classes'], y_pred['classes'])

    # Return a dict mapping metric names to current value
    return {
        'loss': loss_tracker.result(),
        'top1_acc': top1_acc_train.result(),
        'top5_acc': top5_acc_train.result(),
    }

@tf.function
def test_step(data):
    x, y = data

    y_pred = model(x)

    top1_acc_val.update_state(y['classes'], y_pred['classes'])
    top5_acc_val.update_state(y['classes'], y_pred['classes'])

    return {
        'top1_acc': top1_acc_val.result(),
        'top5_acc': top5_acc_val.result(),
    }

### Start training

In [35]:
def adjust_learning_rate(epoch):
    if epoch != 0 and epoch in [41, 61]:
        K.set_value(classifier_optimizer.learning_rate,classifier_optimizer.learning_rate * 0.2)
        K.set_value(feature_extraction_optimizer.learning_rate, feature_extraction_optimizer.learning_rate * 0.2)

In [36]:
def print_metrics(metrics):
    maxlen = max([len(key) for key in metrics.keys()])
    print("\tMetrics:")
    print("\t" + "-" * (maxlen + 1))
    for k, v in metrics.items():
        print(f"\t{k.ljust(maxlen+1)}: {v.numpy():0.2f}")

In [ ]:
NUM_EPOCHS = 50

callbacks.on_train_begin()

logs = None

for epoch in range(1, NUM_EPOCHS + 1):
    print(f'Epoch: {epoch}')
    warm = True if epoch < 20 else False

    # train
    print('Start training')
    callbacks.on_epoch_begin(epoch)

    for step, data in tqdm.tqdm(train_ds.enumerate()):
        callbacks.on_train_batch_begin(step)

        adjust_learning_rate(epoch + 1)
        logs = train_step(data, warm)

        callbacks.on_train_batch_end(step + 1, logs)

        
    loss_tracker.reset_states()
    top1_acc_train.reset_states()
    top5_acc_train.reset_states()
    
    epoch_logs = copy.copy(logs)

    # evaluate
    val_logs = {}

    print('Start evaluating')
    callbacks.on_test_begin()

    for step, data in val_ds.enumerate():
        callbacks.on_test_batch_begin(step)

        val_logs = test_step(data)
        
        callbacks.on_test_batch_end(step)

    callbacks.on_test_end(val_logs)
    
    top1_acc_val.reset_states()
    top5_acc_val.reset_states()

    val_logs = {
        "val_" + name: val for name, val in val_logs.items()
    }
    
    epoch_logs.update(val_logs)

    callbacks.on_epoch_end(epoch, epoch_logs)
    print_metrics(epoch_logs)
    

callbacks.on_train_end(val_logs)

Epoch: 1
Start training


188it [03:21,  1.07s/it]


Start evaluating
	Metrics:
	-------------
	loss         : 5.07
	top1_acc     : 0.03
	top5_acc     : 0.12
	val_top1_acc : 0.08
	val_top5_acc : 0.24
Epoch: 2
Start training


188it [02:02,  1.54it/s]


Start evaluating
	Metrics:
	-------------
	loss         : 3.57
	top1_acc     : 0.20
	top5_acc     : 0.49
	val_top1_acc : 0.13
	val_top5_acc : 0.38
Epoch: 3
Start training


...

188it [02:06,  1.49it/s]


Start evaluating
	Metrics:
	-------------
	loss         : 0.35
	top1_acc     : 0.90
	top5_acc     : 0.99
	val_top1_acc : 0.37
	val_top5_acc : 0.68
Epoch: 34
Start training


188it [02:06,  1.48it/s]


Start evaluating
	Metrics:
	-------------
	loss         : 0.14
	top1_acc     : 0.96
	top5_acc     : 1.00
	val_top1_acc : 0.47
	val_top5_acc : 0.77
Epoch: 35
Start training


149it [01:40,  1.48it/s]

In [ ]:
logs

## Show heatmap

In [ ]:
model.load_weights('/kaggle/working/temp/model.ckpt')

In [ ]:
model.evaluate(val_ds)

In [ ]:
class_ids = [
    "Aeroplane",
    "Bicycle",
    "Bird",
    "Boat",
    "Bottle",
    "Bus",
    "Car",
    "Cat",
    "Chair",
    "Cow",
    "Dining Table",
    "Dog",
    "Horse",
    "Motorbike",
    "Person",
    "Potted Plant",
    "Sheep",
    "Sofa",
    "Train",
    "Tvmonitor",
    "Total",
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))

In [ ]:
for batch_idx, batch in val_ds.enumerate():
    # if batch_idx != 1:
    #     continue

    x, y = batch

    y_pred = model(x, return_cam=True, target=y['classes'])

    cam_per_batch = []


    for i, (image, cams_per_image, prediction, gt) in enumerate(zip(x, y_pred['cam'], y_pred['classes'], y['classes'])):
        print('----------')
        print(i)
        shape = image.shape

        prediction = prediction.numpy()

        prediction_rounded = np.flatnonzero(np.round(prediction))

        if len(prediction_rounded) == 0:
            print('no prediction')
            prediction_rounded = np.argsort(-prediction)[:5]

        print(f'\tPrediction:   {prediction_rounded}')
        print(f'\tScore:        {np.array2string(prediction[prediction_rounded], precision=2, floatmode="fixed")}')
        print(f'\tGround_truth: {np.flatnonzero(gt)}')

        print()

        for target_class, norm_heatmap in cams_per_image.items():
            print(f'\t{target_class}')

            heatmap = cv2.applyColorMap(norm_heatmap, cv2.COLORMAP_JET)
            superimposed = cv2.addWeighted(heatmap, 0.5, image.numpy().astype(np.uint8), 0.5, 0)

            cv2_imshow(superimposed)
    break

## Maxboxaccv2

In [ ]:
model.load_weights('/kaggle/working/temp/model.ckpt')

In [ ]:
SAVED_CAM_PATH = '/kaggle/working/cam'
SPLIT = 'val'

In [ ]:
!rm -rf /kaggle/working/cam

In [ ]:
for batch_idx, batch in val_ds.enumerate():
    # if batch_idx != 1:
    #     continue

    x, y = batch

    y_pred = model(x, return_cam=True, target=y['classes'])

    for i, (image, cams_per_image, prediction, gt, image_id) in enumerate(zip(x, y_pred['cam'], y_pred['classes'], y['classes'], y['id'])):
        shape = image.shape

        prediction = prediction.numpy()

        prediction_rounded = np.flatnonzero(np.round(prediction))

        for target_class, norm_heatmap in cams_per_image.items():
            # Set up path for saving
            image_id = image_id.numpy().decode('ASCII')
            name = os.path.basename(image_id)
            saved_dir = os.path.join(SAVED_CAM_PATH, SPLIT, os.path.dirname(image_id))
            cam_path = os.path.join(saved_dir, name)

            # plt.imshow(norm_heatmap)

            if not os.path.exists(os.path.dirname(cam_path)):
                os.makedirs(os.path.dirname(cam_path))

            # Save min max normalized heatmap
            np.save(cam_path, norm_heatmap)

            # # Superimpose heatmap on original image
            # norm_heatmap = norm_heatmap.astype(np.uint8)
            # heatmap = cv2.applyColorMap(norm_heatmap, cv2.COLORMAP_JET)
            # superimposed = cv2.addWeighted(heatmap, 0.5, image.numpy().astype(np.uint8), 0.5, 0)

            # cv2_imshow(superimposed)
            # cv2.imwrite(f'{cam_path}.png', superimposed)

    #         break
    #     break
    # break

In [ ]:
import glob

len(glob.glob('/kaggle/working/cam/**/*.jpg.npy', recursive=True))

In [ ]:
%cd /kaggle/working/wsolevaluation
!python evaluation.py --scoremap_root=/kaggle/working/cam/val \
                     --metadata_root=metadata/ \
                     --mask_root=dataset/ \
                     --dataset_name=CUB \
                     --split=val \
                     --cam_curve_interval=0.005 \
                     --multi_iou_eval \
                     --multi_contour_eval